In [ ]:
import numpy as np

def RowElimination(x, i, j):
  if j == x.shape[0]:
    x[i, :] =  x[i, :] / x[i, np.flatnonzero(x[i, :])[0]]
    return x

  if x[i, np.flatnonzero(x[i, :])[0]] != 1:
    x[i, :] =  x[i, :] / x[i, np.flatnonzero(x[i, :])[0]]

  x[j, :] = x[j, :] - (x[i, :] * x[j, np.flatnonzero(x[j, :])[0]])
  return x

def CleanMatrix(x):
   col = np.argwhere(np.all(x[..., :] == 0, axis=0))
   row = np.argwhere(np.all(x[:, ...] == 0, axis=1))

   x[:] = np.delete(x, col, axis=1)
   x[:] = np.delete(x, row, axis=0)
   indexes = np.lexsort(np.fliplr(-(np.abs(x))).T)
   return x[indexes]

def GetBasis(x):
  ind = []
  for i in range(x.shape[0]):
    ind.append(np.flatnonzero(x[i, :])[0])
  b = np.delete(x, ind, axis=1)
  free = b[:, -1]
  rest = np.delete(b, -1, axis=1)
  rest = -rest
  basis = x[:, ind]
  inv = np.delete(np.arange(x.shape[1]-1), ind)
  return basis, rest, free, inv

def BackSubstitution(basis, sum):
  basis = basis - np.identity(basis.shape[0])
  for i in range(sum.shape[0]-1, -1, -1):
    basis[:, i] = basis[:, i] * sum[i]
    sum = sum - basis[:,i]
  return sum

def InsertBack(result, ind, subst):
  for i in range(ind.shape[0]):
    result = np.insert(result, ind[i], subst[i])
  return result


def gauss(x):
  for i in range(x.shape[0]):
    x = CleanMatrix(x)
    for j in range(i+1, x.shape[0]+1):
      x = RowElimination(x, i, j)

  basis, rest, free, ind = GetBasis(x)
  result = []

  if rest.shape[1] == 0:
    return BackSubstitution(basis, free)

  for k in range(rest.shape[1]):
    subst = np.zeros(rest.shape[1])
    subst[k] = 1
    sum = np.add((np.dot(rest, subst)).flatten(), free)

    res = BackSubstitution(basis, sum)
    result.append(InsertBack(res, ind, subst))

  return np.array(result)



a1 = np.array([[2, 1, 4], [3, 2, 1], [1, 3 , 3]]).astype(np.float64)
b1 = np.array([16, 10, 16]).astype(np.float64)
test1 = np.hstack((a1, b1[:, np.newaxis]))

print(test1)
print(gauss(test1), '\n')

a2 = np.array([[1, -1, 1, -4], [8, -1, -1, 2], [1, 6, -2, -2]]).astype(np.float64)
b2 = np.array([-2, 11, -7]).astype(np.float64)
test2 = np.hstack((a2, b2[:, np.newaxis]))

print(test2)
print(gauss(test2))

[[ 2.  1.  4. 16.]
 [ 3.  2.  1. 10.]
 [ 1.  3.  3. 16.]]
[1. 2. 3.] 

[[ 1. -1.  1. -4. -2.]
 [ 8. -1. -1.  2. 11.]
 [ 1.  6. -2. -2. -7.]]
[[ 1. -1.  0.  1.]]


In [ ]:
import numpy as np

def GetDiags(x):
  d = x[:, -1]
  temp = np.delete(x, -1, axis=1)
  a = np.diag(temp, k=-1)
  b = np.diag(temp, k=0)
  c = np.diag(temp, k=1)
  return a, b, c, d

def tdma(x):
  a, b, c, d = GetDiags(x)

  n = x.shape[0]
  cnew = np.zeros(n-1).astype(float)
  dnew= np.zeros(n).astype(float)
  res = np.zeros(n).astype(float)
  cnew[0] = c[0]/b[0]
  dnew[0] = d[0]/b[0]

  for i in range(1, n-1):
      cnew[i] = c[i] / (b[i] - a[i-1] * cnew[i-1])
  for j in range(1, n):
      dnew[j] = (d[j] - a[j-1] * dnew[j-1]) / (b[j] - a[j-1] * cnew[j-1])
  res[n-1] = dnew[n-1]
  for k in range(n-1, 0, -1):
      res[k-1] = dnew[k-1] - cnew[k-1] * res[k]

  return res


a1 = np.array([[5, 3, 0, 0], [3, 6, 1, 0], [0, 1, 4, -2], [0, 0, 1, -3]]).astype(np.float64)
b1 = np.array([8, 10, 3, -2]).astype(np.float64)
test1 = np.hstack((a1, b1[:, np.newaxis]))

print(test1)
print(tdma(test1), '\n')

a2 = np.array([[1, 2, 0, 0], [2, -1, 1, 0], [0, 1, -1, 1], [0, 0, 1, 1]]).astype(np.float64)
b2 = np.array([5, 3, 3, 7]).astype(np.float64)
test2 = np.hstack((a2, b2[:, np.newaxis]))

print(test2)
print(tdma(test2), '\n')

a3 = np.array([[2, 1, 0, 0], [2, 3, -1, 0], [0, 1, -1, 3], [0, 0, 1, -1]]).astype(np.float64)
b3 = np.array([4, 9, 12, -4]).astype(np.float64)
test3 = np.hstack((a3, b3[:, np.newaxis]))

print(test3)
print(tdma(test3))

[[ 5.  3.  0.  0.  8.]
 [ 3.  6.  1.  0. 10.]
 [ 0.  1.  4. -2.  3.]
 [ 0.  0.  1. -3. -2.]]
[1. 1. 1. 1.] 

[[ 1.  2.  0.  0.  5.]
 [ 2. -1.  1.  0.  3.]
 [ 0.  1. -1.  1.  3.]
 [ 0.  0.  1.  1.  7.]]
[1. 2. 3. 4.] 

[[ 2.  1.  0.  0.  4.]
 [ 2.  3. -1.  0.  9.]
 [ 0.  1. -1.  3. 12.]
 [ 0.  0.  1. -1. -4.]]
[ 1.  2. -1.  3.]
